In [ ]:
import a1chemy.data_source as data_source
import a1chemy.indicators as indicators
import a1chemy.plot.plotly_tools as plotly_tools
import pymongo
import operator
import pandas as pd
from tqdm.notebook import tqdm
import base64

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client)
mongo_fund_client = data_source.MongoFund(mongo_client)
mongo_fund_client = data_source.MongoFund(mongo_client)

In [ ]:
exchange='SH'
symbol='SH510300'
ticks = mongo_ticks_client.find_one(exchange=exchange, symbol=symbol)
fund = mongo_fund_client.find_one(exchange=exchange, symbol=symbol)

In [ ]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from operator import truediv 


def return_rate(t, day1, day2):
    length = t.raw_data.shape[0]
    if -length >= day1 or -length >= day2:
        return -100
    return "{:.3f}".format(100*(t.close().iloc[day1] - t.close().iloc[day2])/t.close().iloc[day2])






# title=widgets.HBox([widgets.Label(value='Broad Market', layout=layouts[0]), 
#                     widgets.Label(value='SYMBOL', layout=layouts[1]),
#                     widgets.Label(value='Price', layout=layouts[2]),
#                     widgets.Label(value='5D%', layout=layouts[3]), 
#                     widgets.Label(value='1M', layout=layouts[4]), 
#                     widgets.Label(value='shares', layout=layouts[5])])


def relative(n, d, last_count):
    n_data = n.raw_data.iloc[-last_count:None]
    n_last = n_data['close'].tolist()
    d_last = d.raw_data.iloc[-last_count:None]['close'].tolist()
    relative_data = list(map(truediv, n_last, d_last))
    df = pd.DataFrame({'time': n_data['time'].tolist(), 
                       'relative': relative_data})
    return plotly_tools.line_thumbnail(data=df, x_axis='time', y_axis='relative')

def row_data(exchange, symbol, relative_base, layouts):
    ticks = mongo_ticks_client.find_one(exchange=exchange, symbol=symbol)
    fund = mongo_fund_client.find_one(exchange=exchange, symbol=symbol)
    
    count=20
    month_thumbnail = plotly_tools.ticks_thumbnail(ticks=ticks, start=-count)
    shares = plotly_tools.fund_amount_thumbnail(fund=fund, start=-count)
    relative_image_widget = None
    if relative_base.symbol == ticks.symbol:
        relative_image_widget = widgets.Label(value='--'.format(ticks.close().iloc[-1]), layout=layouts[6])
    else:
        relative_image = relative(ticks, relative_base, count)
        relative_image_widget = widgets.Image(value=relative_image.data if relative_image else None, layout=layouts[6])
    return widgets.HBox([widgets.Label(value=ticks.name, layout=layouts[0]), 
                     widgets.Label(value=ticks.symbol, layout=layouts[1]), 
                     widgets.Label(value='{:.3f}'.format(ticks.close().iloc[-1]), layout=layouts[2]), 
                     widgets.Label(value=return_rate(ticks, -1, -6), layout=layouts[3]), 
                     widgets.Image(value=month_thumbnail.data, layout=layouts[4]),
                     widgets.Image(value=shares.data, layout=layouts[5]),
                     relative_image_widget])

def show_rows(symbol_list, relative_base, layouts):
    rows_list = [row_data(d[0], d[1], relative_base, layouts) for d in symbol_list]
    return widgets.VBox(rows_list)

def show_title(data, layouts):
    return widgets.HBox([widgets.Label(value=data[i], layout=layouts[i]) for i in range(len(data))])

In [ ]:
row_layouts = [
    widgets.Layout(width='80px'),
    widgets.Layout(width='90px', display='flex', justify_content="center"),
    widgets.Layout(width='50px', display='flex', justify_content="center"),
    widgets.Layout(width='50px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center")
]

title_layouts = [
    widgets.Layout(width='170px'),
    widgets.Layout(width='50px', display='flex', justify_content="center"),
    widgets.Layout(width='50px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center")
]

board_market_list = [
    ('SH', 'SH515800'),
    ('SH', 'SH510050'),
    ('SH', 'SH510300'),
    ('SH', 'SH510500'),
    ('SZ', 'SZ159915'),
    ('SZ', 'SZ159949')
]
relative_base = mongo_ticks_client.find_one(exchange='SH', symbol='SH515800')

widgets.VBox([
    show_title(['Broad Market', 'Price', '5D%', '1M', 'Shr', 'RM'], title_layouts),
    show_rows(board_market_list, relative_base, row_layouts)])

#layout=widgets.Layout(display='flex', border='solid 1px black', width='641px')